In [ ]:
import math
from scipy.stats import norm
import numpy as np
from datetime import datetime, timedelta

class BlackScholesCalculator:
    """
    Black-Scholes Option Pricing Model Calculator

    The Black-Scholes model calculates the theoretical price of European options
    based on the underlying asset price, strike price, time to expiration,
    risk-free rate, and volatility.
    """

    def __init__(self):
        self.S = None  # Current stock price
        self.K = None  # Strike price
        self.T = None  # Time to expiration (in years)
        self.r = None  # Risk-free rate
        self.sigma = None  # Volatility

    def set_parameters(self, stock_price, strike_price, time_to_expiration,
                      risk_free_rate, volatility):
        """
        Set the Black-Scholes parameters

        Args:
            stock_price (float): Current price of the underlying asset
            strike_price (float): Strike price of the option
            time_to_expiration (float): Time to expiration in years
            risk_free_rate (float): Risk-free interest rate (as decimal, e.g., 0.05 for 5%)
            volatility (float): Volatility of the underlying asset (as decimal, e.g., 0.20 for 20%)
        """
        self.S = stock_price
        self.K = strike_price
        self.T = time_to_expiration
        self.r = risk_free_rate
        self.sigma = volatility

    def d1(self):
        """Calculate d1 parameter"""
        return (math.log(self.S / self.K) + (self.r + 0.5 * self.sigma**2) * self.T) / (self.sigma * math.sqrt(self.T))

    def d2(self):
        """Calculate d2 parameter"""
        return self.d1() - self.sigma * math.sqrt(self.T)

    def call_price(self):
        """
        Calculate European call option price using Black-Scholes formula

        Returns:
            float: Theoretical call option price
        """
        if any(param is None for param in [self.S, self.K, self.T, self.r, self.sigma]):
            raise ValueError("All parameters must be set before calculating option price")

        if self.T <= 0:
            # Option has expired
            return max(self.S - self.K, 0)

        d1_val = self.d1()
        d2_val = self.d2()

        call_price = (self.S * norm.cdf(d1_val) -
                     self.K * math.exp(-self.r * self.T) * norm.cdf(d2_val))

        return call_price

    def put_price(self):
        """
        Calculate European put option price using Black-Scholes formula

        Returns:
            float: Theoretical put option price
        """
        if any(param is None for param in [self.S, self.K, self.T, self.r, self.sigma]):
            raise ValueError("All parameters must be set before calculating option price")

        if self.T <= 0:
            # Option has expired
            return max(self.K - self.S, 0)

        d1_val = self.d1()
        d2_val = self.d2()

        put_price = (self.K * math.exp(-self.r * self.T) * norm.cdf(-d2_val) -
                    self.S * norm.cdf(-d1_val))

        return put_price

    def greeks(self):
        """
        Calculate the Greeks (risk sensitivities)

        Returns:
            dict: Dictionary containing Delta, Gamma, Theta, Vega, and Rho
        """
        if any(param is None for param in [self.S, self.K, self.T, self.r, self.sigma]):
            raise ValueError("All parameters must be set before calculating Greeks")

        if self.T <= 0:
            return {"Delta": 0, "Gamma": 0, "Theta": 0, "Vega": 0, "Rho": 0}

        d1_val = self.d1()
        d2_val = self.d2()

        # Delta (price sensitivity to underlying asset price)
        call_delta = norm.cdf(d1_val)
        put_delta = call_delta - 1

        # Gamma (delta sensitivity to underlying asset price)
        gamma = norm.pdf(d1_val) / (self.S * self.sigma * math.sqrt(self.T))

        # Theta (price sensitivity to time decay) - per day
        call_theta = ((-self.S * norm.pdf(d1_val) * self.sigma / (2 * math.sqrt(self.T))) -
                     (self.r * self.K * math.exp(-self.r * self.T) * norm.cdf(d2_val))) / 365

        put_theta = ((-self.S * norm.pdf(d1_val) * self.sigma / (2 * math.sqrt(self.T))) +
                    (self.r * self.K * math.exp(-self.r * self.T) * norm.cdf(-d2_val))) / 365

        # Vega (price sensitivity to volatility)
        vega = self.S * norm.pdf(d1_val) * math.sqrt(self.T) / 100  # Per 1% volatility change

        # Rho (price sensitivity to risk-free rate)
        call_rho = self.K * self.T * math.exp(-self.r * self.T) * norm.cdf(d2_val) / 100
        put_rho = -self.K * self.T * math.exp(-self.r * self.T) * norm.cdf(-d2_val) / 100

        return {
            "Call_Delta": call_delta,
            "Put_Delta": put_delta,
            "Gamma": gamma,
            "Call_Theta": call_theta,
            "Put_Theta": put_theta,
            "Vega": vega,
            "Call_Rho": call_rho,
            "Put_Rho": put_rho
        }

def days_to_expiration(expiration_date):
    """
    Convert expiration date to time in years

    Args:
        expiration_date (str): Expiration date in YYYY-MM-DD format

    Returns:
        float: Time to expiration in years
    """
    exp_date = datetime.strptime(expiration_date, '%Y-%m-%d')
    today = datetime.now()
    days_diff = (exp_date - today).days
    return days_diff / 365.25

def main():
    """Main function to run the Black-Scholes calculator"""
    print("=" * 60)
    print("Black-Scholes Option Pricing Calculator")
    print("=" * 60)

    calculator = BlackScholesCalculator()

    try:
        # Get user input
        print("\nPlease enter the following parameters:")

        stock_price = float(input("Current stock price ($): "))
        strike_price = float(input("Strike price ($): "))

        # Time to expiration input
        time_input_method = input("Enter time to expiration as (1) days or (2) date (YYYY-MM-DD)? [1/2]: ")

        if time_input_method == "2":
            exp_date = input("Expiration date (YYYY-MM-DD): ")
            time_to_exp = days_to_expiration(exp_date)
        else:
            days = float(input("Days to expiration: "))
            time_to_exp = days / 365.25

        risk_free_rate = float(input("Risk-free rate (as %, e.g., 5 for 5%): ")) / 100
        volatility = float(input("Volatility (as %, e.g., 20 for 20%): ")) / 100

        # Set parameters
        calculator.set_parameters(stock_price, strike_price, time_to_exp,
                                risk_free_rate, volatility)

        # Calculate option prices
        call_price = calculator.call_price()
        put_price = calculator.put_price()

        # Calculate Greeks
        greeks = calculator.greeks()

        # Display results
        print("\n" + "=" * 60)
        print("OPTION PRICING RESULTS")
        print("=" * 60)

        print(f"\nInput Parameters:")
        print(f"  Stock Price:      ${stock_price:.2f}")
        print(f"  Strike Price:     ${strike_price:.2f}")
        print(f"  Time to Exp:      {time_to_exp:.4f} years ({time_to_exp*365.25:.0f} days)")
        print(f"  Risk-free Rate:   {risk_free_rate:.2%}")
        print(f"  Volatility:       {volatility:.2%}")

        print(f"\nOption Prices:")
        print(f"  Call Option:      ${call_price:.4f}")
        print(f"  Put Option:       ${put_price:.4f}")

        print(f"\nThe Greeks:")
        print(f"  Call Delta:       {greeks['Call_Delta']:.4f}")
        print(f"  Put Delta:        {greeks['Put_Delta']:.4f}")
        print(f"  Gamma:            {greeks['Gamma']:.4f}")
        print(f"  Call Theta:       ${greeks['Call_Theta']:.4f} per day")
        print(f"  Put Theta:        ${greeks['Put_Theta']:.4f} per day")
        print(f"  Vega:             ${greeks['Vega']:.4f} per 1% vol change")
        print(f"  Call Rho:         ${greeks['Call_Rho']:.4f} per 1% rate change")
        print(f"  Put Rho:          ${greeks['Put_Rho']:.4f} per 1% rate change")

        # Additional analysis
        print(f"\n" + "=" * 60)
        print("ADDITIONAL ANALYSIS")
        print("=" * 60)

        moneyness = stock_price / strike_price
        if moneyness > 1.05:
            print(f"The call option is IN-THE-MONEY (S/K = {moneyness:.3f})")
        elif moneyness < 0.95:
            print(f"The call option is OUT-OF-THE-MONEY (S/K = {moneyness:.3f})")
        else:
            print(f"The call option is AT-THE-MONEY (S/K = {moneyness:.3f})")

        # Intrinsic vs Time value
        call_intrinsic = max(stock_price - strike_price, 0)
        call_time_value = call_price - call_intrinsic

        print(f"\nCall Option Breakdown:")
        print(f"  Intrinsic Value:  ${call_intrinsic:.4f}")
        print(f"  Time Value:       ${call_time_value:.4f}")

    except ValueError as e:
        print(f"Error: {e}")
        print("Please ensure all inputs are valid numbers.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage with preset data
def example_calculation():
    """Example calculation with sample data"""
    print("\n" + "=" * 60)
    print("EXAMPLE CALCULATION")
    print("=" * 60)

    calculator = BlackScholesCalculator()

    # Example: Apple stock option
    calculator.set_parameters(
        stock_price=150.0,    # Current AAPL price
        strike_price=155.0,   # Strike price
        time_to_expiration=30/365.25,  # 30 days to expiration
        risk_free_rate=0.05,  # 5% risk-free rate
        volatility=0.25       # 25% volatility
    )

    call_price = calculator.call_price()
    put_price = calculator.put_price()

    print(f"Example: AAPL $155 Call/Put expiring in 30 days")
    print(f"Stock: $150, Strike: $155, 30 days, 5% rate, 25% vol")
    print(f"Call Price: ${call_price:.4f}")
    print(f"Put Price: ${put_price:.4f}")

if __name__ == "__main__":
    main()

    # Uncomment the line below to see an example calculation
    # example_calculation()

Black-Scholes Option Pricing Calculator

Please enter the following parameters:
Current stock price ($): 23
Strike price ($): 28
Enter time to expiration as (1) days or (2) date (YYYY-MM-DD)? [1/2]: 1
Days to expiration: 21
Risk-free rate (as %, e.g., 5 for 5%): 5
Volatility (as %, e.g., 20 for 20%): 20

OPTION PRICING RESULTS

Input Parameters:
  Stock Price:      $23.00
  Strike Price:     $28.00
  Time to Exp:      0.0575 years (21 days)
  Risk-free Rate:   5.00%
  Volatility:       20.00%

Option Prices:
  Call Option:      $0.0000
  Put Option:       $4.9196

The Greeks:
  Call Delta:       0.0000
  Put Delta:        -1.0000
  Gamma:            0.0001
  Call Theta:       $-0.0000 per day
  Put Theta:        $0.0038 per day
  Vega:             $0.0000 per 1% vol change
  Call Rho:         $0.0000 per 1% rate change
  Put Rho:          $-0.0161 per 1% rate change

ADDITIONAL ANALYSIS
The call option is OUT-OF-THE-MONEY (S/K = 0.821)

Call Option Breakdown:
  Intrinsic Value:  $0.000